In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os, argparse, math, random
from datetime import datetime

testFiles = []
ka_test_file = "../dataset/prep/codebook/test_c0_"
kAA_test_file = "../dataset/prep/codebook/test_c1_"
kha_test_file = "../dataset/prep/codebook/test_c2_"

ka_train_file = "../dataset/prep/codebook/train_c0_"
kAA_train_file = "../dataset/prep/codebook/train_c1_"
kha_train_file = "../dataset/prep/codebook/train_c2_"

fileName = "../dataset/prep/codebook/train_c"

outputPath = "../output/abpi/"

# for every m and n combination (m = 2,4,8,16,32) and (n = 2,3,4,5)

# N = [2,3,4,5]
# M = [4,8,16,32]

N = [2]
M = [4]

wholeData = []

A = []
B = []
PI = []

In [2]:
def fileHandle(fileName):
    global wholeData
    wholeData = []
    file = open(fileName)
    for line in file:
        teLine = line.rstrip('\n ').split(' ')
        nLine = [int(i) for i in teLine]
        nLine = np.array(nLine)
#         print(nLine)
        wholeData.append(nLine)
    file.close()

def writeFile(classIndex, name, values, n, m, ext):
    targetPath = outputPath + name + "_c" + str(classIndex) + "_n" + str(n) + "_m" + str(m) + ext

    if not os.path.exists(os.path.dirname(targetPath)):
        try:
            os.makedirs(os.path.dirname(targetPath))
        except OSError as exc:  # Guard against race condition
            if exc.errno != errno.EEXIST:
                raise
    try:
        print("target File: ", targetPath)
        outfile = open(targetPath, "w")
    except IOError:
        print("File not created !!!!!!!!!!!!!!!!!!!!!!!!!")

    for value in values:
        for feature in value:
            outfile.write(str(feature)+" ")
        outfile.write("\n")
    outfile.close()
    print("Done !")

In [3]:
def EMAlgo(n, m):
    global A
    global B
    global PI
    global wholeData
    
    probI = 1e9
    probF = 0
    
    while abs(probI - probF) >= 1e-3:
        
        newA = np.zeros((n, n))
        newB = np.zeros((n, m))
        newPI = np.zeros((n))
        
#         print(B)
        
        # remind to send old - A, B in functions(forward, backward)
        # and calc of zeta, gamma
        # also remember to equate A = newA at the end of following for-loop!!
        
        probI = probF
                
        for obs in wholeData:
            
            tempPLog = 0
            # From here, E-step start!!
            # See for alpha, beta to be set
            alpha = forwdProc(obs, n)
            beta = backwdProc(obs, n)
            
#             print(alpha)
            
            lObs = len(obs)
            
            for i in range(n):
                tempPLog += alpha[lObs-1,i]
            
            zeta = np.zeros((lObs-1, n, n))
                
            # pass the n as parameter in the func EMAlgo itself
            
            # common denominator
            deno = np.zeros((lObs-1))
            
            for ti in range(lObs-1):
                for i in range(n):
                    for j in range(n):
                        deno[ti] += alpha[ti,i] * A[i,j] * B[j,obs[ti+1]] * beta[ti+1,j]
                        
            
            for ti in range(lObs-1):
                for i in range(n):
                    for j in range(n):
                        zeta[ti,i,j] = alpha[ti,i] * A[i,j] * B[j,obs[ti+1]] * beta[ti+1,j]
                        zeta[ti,i,j] /= deno[ti]
                        
            
            
            gamma = np.zeros((lObs, n))
            deno = np.zeros((lObs))
            
            for ti in range(lObs):
                for i in range(n):
                    deno[ti] += alpha[ti,i] * beta[ti,i]
                    
            
            for ti in range(lObs):
                for i in range(n):
                    gamma[ti,i] = alpha[ti,i] * beta[ti,i]
                    gamma[ti,i] /= deno[ti]
                    
            
            # E-step ends
            
            # M-step begins
            
            a = np.zeros((n, n))
            b = np.zeros((n, m))
            pi = np.zeros((n))
            
            for i in range(n):
                pi[i] = gamma[0,i]
                
            
            for i in range(n):
                denoGamma = 0
                nrtZeta = 0
                
                for ti in range(lObs-1):
                    denoGamma += gamma[ti,i]
                
                for j in range(n):
                    for ti in range(lObs-1):
                        nrtZeta += zeta[ti,i,j]
                    
                    a[i,j] = nrtZeta / denoGamma
                    
            
            # loop over the observation sequence!!

            for ti,syb in enumerate(obs):

                for j in range(n):

                    b[j,int(syb)] += gamma[ti,j]


            for j in range(n):
                tempVal = 0
                for ti in range(lObs):
                    tempVal += gamma[ti,j]

                for k in range(m):
                    b[j,k] /= tempVal


            newA += a
            newB += b
            newPI += pi
            

            probF += np.log(tempPLog)
            
        lData = len(wholeData)
        
#         print(lData)
        
        A = newA / lData
        B = newB / lData
        PI = newPI / lData
    

In [4]:
def initialize(n, m, classIndex):
    # call to EM from here!!
    # for each class loop
    # read the file from here itself for every class separately loop on the class itself 
    # outer loop for the class
    
    global A
    global B
    global PI
    global wholeData
    
    #treat them as global!!
    A = np.zeros((n,n))
    B = np.zeros((n,m))
    PI = np.zeros((n))
    
    for obs in wholeData:
        # obs is my numpy array
        
        a = np.zeros((n,n))
        b = np.zeros((n,m))
        
        lgt = len(obs)
        gone = int(lgt / n)
        toFirst = lgt - gone * n
        toFirst += gone
                
        for i in range(n-1):
            if i == 0:
                a[i,i] = (toFirst - 1) / toFirst
                a[i,i+1] = 1 / toFirst
            elif gone != 0:
                a[i,i] = (gone - 1) / gone
                a[i,i+1] = 1 / gone
        
        if gone != 0:
            a[n-1][n-1] = (gone - 1) / gone
            
        j = 0
        anth = 0
        for i,syb in enumerate(obs):
            if j == 0 and i < toFirst:
                b[0,int(syb)] += 1
                anth += 1
                continue
            elif j == 0:
                j = 1
                anth = 0
                
            if anth >= gone:
                j += 1
                anth = 0
            
            b[j,int(syb)] += 1
            anth += 1
            
        for i in range(n):
            if i == 0:
                b[i] /= toFirst
            elif gone != 0:
                b[i] /= gone
                
        A += a
        B += b
        
        
    lData = len(wholeData)
    A /= lData
    B /= lData
    PI[0] = 1
    
    #return from EMAlgo A, B, PI
    EMAlgo(n, m)    
    
    writeFile(classIndex, "em", A, n, m, ".a")
    writeFile(classIndex, "em", B, n, m, ".b")
    writeFile(classIndex, "em", [PI], n, m, ".pi")
    

In [5]:
def forwdProc(obs, n):
    
    global A
    global B
    global PI
    global wholeData
    
    # Assuming I know n
    lObs = len(obs)
    
    alpha = np.zeros((lObs, n))
    for i in range(n):
        alpha[0,i] = PI[i] * B[i,obs[0]]
        
    for ti in range(1,lObs):
        for j in range(n):
            storeVal = 0
            for i in range(n):
                storeVal += alpha[ti-1,i] * A[i,j]
            alpha[ti,j] = storeVal * B[j, obs[ti]]
    
    return alpha

In [6]:
def backwdProc(obs, n):
    # assuming I know n
    
    global A
    global B
    global PI
    global wholeData
    
    lObs = len(obs)
    beta = np.zeros((lObs, n))
    
    for i in range(n):
        beta[lObs-1,i] = 1
    
    for ti in range(lObs-2, -1, -1):
        for i in range(n):
            storeVal = 0
            for j in range(n):
                storeVal += A[i,j] * B[j,obs[ti+1]] * beta[ti+1,j]
            beta[ti,i] = storeVal
    
    return beta

In [7]:
for n in N:
    for m in M:
        # now read the files and store the data in wholeData
        # list of numpy arrays - wholeData[0] = one observation sequence, one numpy array
        print("M: ", m)
        for i in range(3):
            fileHandle(fileName+str(i)+"_"+ str(m) + ".cb")
            initialize(n,m,i);

M:  4


KeyboardInterrupt: 